# email_module

The email module is part of Python’s standard library and is used to create, parse, and manipulate email messages compliant with Internet email standards (RFC 5322, MIME).

It does not send emails by itself; instead, it builds email messages that are typically sent using the smtplib module.

The email module is used to construct and parse MIME-compliant email messages.

### **`email`**  builds the message 👊, **`smtplib`**  sends it. 📮

Google chat Webhook 

In [1]:
import json
import os
import socket
import urllib.request
from datetime import datetime
from typing import Optional


class GoogleChatNotifier:
    """
    Sends operational notifications to Google Chat via webhook.
    """

    def __init__(self, webhook_url: Optional[str] = None):
        self.webhook_url = webhook_url or os.getenv("GOOGLE_CHAT_WEBHOOK_URL")
        if not self.webhook_url:
            raise ValueError("Google Chat webhook URL not provided")

    def send(
            self,
            title: str,
            message: str,
            status: str,
            job_name: str,
            environment: str = "prod",
            extra: dict | None = None,
    ):
        payload = {
            "cards": [
                {
                    "header": {
                        "title": title,
                        "subtitle": f"{job_name} | {environment.upper()}",
                    },
                    "sections": [
                        {
                            "widgets": [
                                {
                                    "keyValue": {
                                        "topLabel": "Status",
                                        "content": status,
                                    }
                                },
                                {
                                    "keyValue": {
                                        "topLabel": "Host",
                                        "content": socket.gethostname(),
                                    }
                                },
                                {
                                    "keyValue": {
                                        "topLabel": "Timestamp",
                                        "content": datetime.utcnow().isoformat() + "Z",
                                    }
                                },
                                {
                                    "textParagraph": {
                                        "text": message
                                    }
                                },
                            ]
                        }
                    ],
                }
            ]
        }

        if extra:
            payload["cards"][0]["sections"][0]["widgets"].append(
                {
                    "textParagraph": {
                        "text": f"<b>Details:</b><br/>{json.dumps(extra, indent=2)}"
                    }
                }
            )

        request = urllib.request.Request(
            self.webhook_url,
            data=json.dumps(payload).encode("utf-8"),
            headers={"Content-Type": "application/json"},
            method="POST",
        )

        with urllib.request.urlopen(request, timeout=10) as response:
            if response.status != 200:
                raise RuntimeError(
                    f"Google Chat webhook failed: {response.status}"
                )


start

In [ ]:

notifier = GoogleChatNotifier()

notifier.send(
    title="AWS Batch Job Started",
    message="Batch job execution has started.",
    status="STARTED",
    job_name="daily_sales_etl",
    environment="prod",
)

success

In [ ]:
notifier.send(
    title="AWS Batch Job Completed",
    message="Batch job completed successfully.",
    status="SUCCESS",
    job_name="daily_sales_etl",
    environment="prod",
    extra={
        "records_processed": 2_340_912,
        "duration_minutes": 18.4,
    },
)

fail

In [ ]:
try:
    # run_job()
    ...
except Exception as exc:
    notifier.send(
        title="AWS Batch Job Failed",
        message=str(exc),
        status="FAILED",
        job_name="daily_sales_etl",
        environment="prod",
        extra={
            "error_type": type(exc).__name__,
        },
    )
    raise

### Store the webhook url in Cloud Secrets Manager or AWS Secrets Manager for better security.